In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('../datasets/spam.csv', encoding = "ISO-8859-1")
df = df.iloc[:,0:2]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [2]:
print(sum(df.loc[:,"v1"]=="ham"))
print(sum(df.loc[:,"v1"]=="spam"))

4825
747


In [3]:
#%pip install -U spacy
#%pip install -U spacy-lookups-data
#!python -m spacy download en_core_web_sm
import spacy
from spacy.lookups import Lookups
nlp = spacy.load("en_core_web_sm")

In [4]:
def textPocess(sms):
    sms = sms.lower()
    sms = nlp(sms)
    my_stopwords = nlp.Defaults.stop_words
    sms = [token.lemma_ for token in sms if token.lower_ not in my_stopwords]
    sms = ' '.join(sms)
    return sms

In [5]:
textPocess('I love coding and writing')

'love code write'

In [6]:
df['v2'] = df['v2'].map(lambda x: textPocess(x))

In [7]:
df['v2']

0       jurong point , crazy .. available bugis n grea...
1                           ok lar ... joke wif u oni ...
2       free entry 2 wkly comp win fa cup final tkts 2...
3                             u dun early hor ... u c ...
4                                 nah think go usf , live
                              ...                        
5567    2nd time try 2 contact u. u win å£750 pound pr...
5568                         ì _ b go esplanade fr home ?
5569                     pity , * mood . ... suggestion ?
5570    guy bitching act like interested buy week give...
5571                                          rofl . true
Name: v2, Length: 5572, dtype: object

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()
df_new = count_vectorizer.fit_transform(df['v2'])

In [42]:
count_vectorizer.vocabulary_

{'jurong': 3837,
 'point': 5195,
 'crazy': 2095,
 'available': 1223,
 'bugis': 1611,
 'great': 3207,
 'world': 7347,
 'la': 3945,
 'buffet': 1609,
 'cine': 1865,
 'get': 3088,
 'amore': 1025,
 'wat': 7169,
 'ok': 4835,
 'lar': 3978,
 'joke': 3803,
 'wif': 7268,
 'oni': 4861,
 'free': 2957,
 'entry': 2626,
 'wkly': 7313,
 'comp': 1966,
 'win': 7280,
 'fa': 2746,
 'cup': 2148,
 'final': 2838,
 'tkts': 6717,
 '21st': 412,
 '2005': 403,
 'text': 6592,
 '87121': 778,
 'receive': 5520,
 'question': 5433,
 'std': 6281,
 'txt': 6878,
 'rate': 5477,
 'apply': 1102,
 '08452810075over18': 77,
 'dun': 2508,
 'early': 2524,
 'hor': 3447,
 'nah': 4590,
 'think': 6648,
 'go': 3129,
 'usf': 7006,
 'live': 4092,
 'freemsg': 2964,
 'hey': 3375,
 'darle': 2198,
 'week': 7207,
 'word': 7339,
 'like': 4058,
 'fun': 3011,
 'tb': 6534,
 'xxx': 7404,
 'chgs': 1822,
 'send': 5867,
 '50': 604,
 'rcv': 5487,
 'brother': 1585,
 'speak': 6186,
 'treat': 6820,
 'aids': 960,
 'patent': 5023,
 'request': 5604,
 'mell

In [43]:
df_new = pd.DataFrame(df_new.toarray(), columns=count_vectorizer.get_feature_names_out())
df_new.head()

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,ìï,ó_,û_thanks,ûªm,ûªt,ûªve,ûï,ûïharry,ûò,ûówell
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df['v1'] = df['v1'].map(lambda x: 1 if x == 'spam' else 0)

In [44]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_classif, y_classif = ros.fit_resample(df_new, df['v1'])

In [45]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
def check_classif(X, y):
    X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)
    lr = LogisticRegression(random_state=42, C=0.5, n_jobs=-1).fit(X_train, y_train)
    y_pred = lr.predict(X_test)
    print(classification_report(y_test, y_pred))

In [46]:
check_classif(df_new, df['v1']) #до оверсемплинга

              precision    recall  f1-score   support

           0       0.97      1.00      0.99       965
           1       1.00      0.83      0.91       150

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [47]:
check_classif(X_classif, y_classif) #после 

              precision    recall  f1-score   support

           0       1.00      0.99      1.00       985
           1       0.99      1.00      1.00       945

    accuracy                           1.00      1930
   macro avg       1.00      1.00      1.00      1930
weighted avg       1.00      1.00      1.00      1930



In [50]:
import sys
sys.path.append( 'C:\Machine Learning\mylib' )
sys.path.append( 'L:\ML\mylib' )
import PCA as Pca
p = Pca.PCA(1000)
new_data = p.fit_transform(X_classif)

In [51]:
check_classif(new_data, y_classif) #на 500 около 0.8

              precision    recall  f1-score   support

           0       0.99      0.84      0.91       985
           1       0.85      0.99      0.92       945

    accuracy                           0.91      1930
   macro avg       0.92      0.92      0.91      1930
weighted avg       0.92      0.91      0.91      1930



In [52]:
from sklearn.decomposition import LatentDirichletAllocation

In [53]:
tf_vectorizer = CountVectorizer(max_features=1000, stop_words='english')
tf = tf_vectorizer.fit_transform(df['v2'])

In [54]:
lda = LatentDirichletAllocation(n_components=2, max_iter=20, 
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0).fit(tf)

In [55]:
lda.components_.shape

(2, 1000)

In [56]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [59]:
print_top_words(lda, tf_vectorizer.get_feature_names_out(), 30)

Topic #0:
come know good day time love like want think need tell sorry ok home work later ask say hope miss night great leave thing meet pls today happy dear feel
Topic #1:
gt lt ur free send text txt stop reply mobile win da ok phone lor hi message min claim msg www prize new week wat cash tone buy service uk
